In [ ]:
# Install necessary packages
!pip install geemap earthengine-api

import ee
import geemap
import os
import shutil
import zipfile
from datetime import datetime, timedelta

# Authenticate with the service account
service_account_key = '/kaggle/input/earth-engine-key/ee-drought-forecasting-d17f1e4a39f6.json'
credentials = ee.ServiceAccountCredentials('kaggle@ee-drought-forecasting.iam.gserviceaccount.com', service_account_key)
ee.Initialize(credentials)

# Define the CHIRPS dataset
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")

# Define the list of countries for West Africa
region_countries = [
    "Mauritania", "Mali", "Niger", "Nigeria", "Chad",
    "Burkina Faso", "Benin", "Togo", "Ghana",
    "Cote d'Ivoire", "Senegal", "Guinea",
    "Guinea-Bissau", "Sierra Leone", "Liberia",
    "Gambia, The", "Sudan"
]

# Use the USDOS/LSIB_SIMPLE/2017 dataset for country boundaries
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Directory in Kaggle to save .tif files
save_dir = '/kaggle/working/SPI_Images'
os.makedirs(save_dir, exist_ok=True)

# Function to sanitize country names for safe filenames
def sanitize_country_name(country_name):
    return country_name.replace("'", "").replace(" ", "_")

# Function to calculate monthly precipitation
def get_monthly_precipitation(year):
    monthly_precipitation = []
    for month in range(1, 13):
        sum_precipitation = chirps \
            .filter(ee.Filter.calendarRange(year, year, 'year')) \
            .filter(ee.Filter.calendarRange(month, month, 'month')) \
            .select('precipitation') \
            .sum() \
            .rename('precipitation_sum')
        monthly_precipitation.append(sum_precipitation)
    return ee.ImageCollection(monthly_precipitation)

# Function to calculate SPI for a country, year, and month
def calculate_spi(country_geometry, year, month):
    monthly_collection = get_monthly_precipitation(year)
    mean = monthly_collection.mean().rename('mean')
    std_dev = monthly_collection.reduce(ee.Reducer.stdDev()).rename('stdDev')

    current_month_precipitation = chirps \
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(month, month, 'month')) \
        .select('precipitation') \
        .sum() \
        .rename('current_month_sum')

    spi = current_month_precipitation.subtract(mean).divide(std_dev).rename('SPI')
    return spi.clip(country_geometry)

# Process SPI for each country
for country_name in region_countries:
    print(f"Processing SPI for {country_name}...")
    
    # Sanitize country name for file paths
    safe_country_name = sanitize_country_name(country_name)
    
    # Create a separate folder for the country
    country_save_dir = os.path.join(save_dir, safe_country_name)
    os.makedirs(country_save_dir, exist_ok=True)
    
    # Get the geometry for the country
    country = countries.filter(ee.Filter.eq('country_na', country_name))
    geometry = country.geometry()
    
    # Process each year and month
    for year in range(2000, 2025):
        for month in range(1, 13):
            spi_image = calculate_spi(geometry, year, month)
            description = f"{safe_country_name}_SPI_{year}_{month:02d}"
            file_path = os.path.join(country_save_dir, f"{description}.tif")
            
            # Export SPI image
            geemap.ee_export_image(
                spi_image,
                filename=file_path,
                scale=1000,  # Adjust scale as needed for CHIRPS resolution
                region=geometry.bounds().getInfo()['coordinates'],
                file_per_band=False
            )
            print(f"Exported: {description}")
    
    # Compress the folder into a ZIP file
    zip_file_path = f"/kaggle/working/{safe_country_name}.zip"
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(country_save_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, country_save_dir)
                zipf.write(file_path, arcname)
    
    # Delete the country folder to save space
    shutil.rmtree(country_save_dir)
    print(f"SPI data for {country_name} saved and compressed as {zip_file_path}.")